In [485]:
import numpy as np
import pandas as pd
import tensorflow as tf
import torch
from torch import nn
from sentence_transformers import SentenceTransformer, util
from tensorflow.keras.applications.vgg16 import VGG16

In [486]:
df = pd.read_csv(r"labels.csv")
df.head()

,Unnamed: 0,image_name,text_ocr,text_corrected,humour,sarcasm,offensive,motivational,overall_sentiment
0,0,image_1.jpg,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,hilarious,general,not_offensive,not_motivational,very_positive
1,1,image_2.jpeg,The best of #10 YearChallenge! Completed in le...,The best of #10 YearChallenge! Completed in le...,not_funny,general,not_offensive,motivational,very_positive
2,2,image_3.JPG,Sam Thorne @Strippin ( Follow Follow Saw every...,Sam Thorne @Strippin ( Follow Follow Saw every...,very_funny,not_sarcastic,not_offensive,not_motivational,positive
3,3,image_4.png,10 Year Challenge - Sweet Dee Edition,10 Year Challenge - Sweet Dee Edition,very_funny,twisted_meaning,very_offensive,motivational,positive
4,4,image_5.png,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,hilarious,very_twisted,very_offensive,not_motivational,neutral


In [487]:
df = df.iloc[: , 1:]
df.head()

,image_name,text_ocr,text_corrected,humour,sarcasm,offensive,motivational,overall_sentiment
0,image_1.jpg,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,hilarious,general,not_offensive,not_motivational,very_positive
1,image_2.jpeg,The best of #10 YearChallenge! Completed in le...,The best of #10 YearChallenge! Completed in le...,not_funny,general,not_offensive,motivational,very_positive
2,image_3.JPG,Sam Thorne @Strippin ( Follow Follow Saw every...,Sam Thorne @Strippin ( Follow Follow Saw every...,very_funny,not_sarcastic,not_offensive,not_motivational,positive
3,image_4.png,10 Year Challenge - Sweet Dee Edition,10 Year Challenge - Sweet Dee Edition,very_funny,twisted_meaning,very_offensive,motivational,positive
4,image_5.png,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,hilarious,very_twisted,very_offensive,not_motivational,neutral


In [488]:
df.shape

(6992, 8)

In [489]:
df['overall_sentiment'].mask(df['overall_sentiment'] == 'positive', 1, inplace=True)
df['overall_sentiment'].mask(df['overall_sentiment'] == 'very_positive', 1, inplace=True)
df['overall_sentiment'].mask(df['overall_sentiment'] == 'negative', 2, inplace=True)
df['overall_sentiment'].mask(df['overall_sentiment'] == 'very_negative', 2, inplace=True)
df['overall_sentiment'].mask(df['overall_sentiment'] == 'neutral', 0, inplace=True)

In [490]:
df['overall_sentiment'].value_counts()

1    4160
0    2201
2     631
Name: overall_sentiment, dtype: int64

In [491]:
Duplicate_negative_Sentiment_rows=df[df['overall_sentiment']==2]
Duplicate_negative_Sentiment_rows.shape

(631, 8)

In [492]:
for i in range(10):
    df = df.sample(frac = 1)

In [493]:
df.dropna(axis=0, how="any", thresh=None, subset=None, inplace=True)

In [494]:
df.isnull().sum()

image_name           0
text_ocr             0
text_corrected       0
humour               0
sarcasm              0
offensive            0
motivational         0
overall_sentiment    0
dtype: int64

# Text Embedding

In [495]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['text_corrected'])
def embedding_for_vocab(filepath, word_index,embedding_dim):
    vocab_size = len(word_index) + 1
    embedding_matrix_vocab = np.zeros((vocab_size,embedding_dim))

    with open(filepath, encoding="utf8") as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word]
                embedding_matrix_vocab[idx] = np.array(vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix_vocab
embedding_dim = 50
embedding_matrix_vocab = embedding_for_vocab("E:\glove.6B.300d.txt\glove.6B.300d.txt", tokenizer.word_index,embedding_dim)

print("Dense vector for first word is => ",embedding_matrix_vocab[1])

Dense vector for first word is =>  [ 0.04656     0.21318001 -0.0074364  -0.45853999 -0.035639    0.23643
 -0.28836     0.21521001 -0.13485999 -1.64129996 -0.26091     0.032434
  0.056621   -0.043296   -0.021672    0.22476    -0.075129   -0.067018
 -0.14247     0.038825   -0.18951     0.29977     0.39304999  0.17887001
 -0.17343    -0.21178     0.23616999 -0.063681   -0.42318001 -0.11661
  0.093754    0.17296    -0.33072999  0.49112001 -0.68994999 -0.092462
  0.24742    -0.17991     0.097908    0.083118    0.15299    -0.27276
 -0.038934    0.54452997  0.53737003  0.29104999 -0.0073514   0.04788
 -0.40759999 -0.026759  ]


In [496]:
text_model = SentenceTransformer('stsb-distilbert-base')
text_model.max_seq_length = 128

In [497]:
def Calulate_textEmbeddings(model,text):
    return model.encode(text, convert_to_tensor=True)

# Image Embedding

In [498]:
img_model = VGG16(input_shape = (224, 224, 3),include_top = False,weights = 'imagenet')

In [499]:
def Calulate_imageEmbeddings(model,imagePath):   
    image = tf.keras.preprocessing.image.load_img(imagePath,target_size=(224,224))
    input_arr = tf.keras.preprocessing.image.img_to_array(image)
    input_arr = np.array([input_arr])
    img_embeddings = model(input_arr)
    meanImgEmb1 = np.mean(img_embeddings,axis =0)
    meanImgEmb = np.mean(meanImgEmb1,axis=0)
    return meanImgEmb

# Concatenating Image and Text Embeddings

In [500]:
# text_embeddings={}
# image_embeddings={}
# cembList=[]
# start_time = time.time()
# i=1
# for index,row in  df.iterrows():
#     text_embeddings[row[0]] = get_textEmbeddings(text_model,str(row[1]))
#     image_embeddings[row[0]] = get_imageEmbeddings(img_model,r'C:\Users\MOON COMPUTER\Desktop\images'+'\\'+row[0])
#     cmb_emb = np.concatenate((text_embeddings[row[0]],image_embeddings[row[0]]),axis=0)
#     norm = np.linalg.norm(cmb_emb)
#     cmb_emb_normal = cmb_emb/norm
#     cembList.append(cmb_emb_normal)
#     print(i)
#     i+=1
# np.savetxt('combinedEmbedding1.txt',cembList)

In [501]:
cembList = np.loadtxt('combinedEmbedding1.txt')
cembList

array([[-0.00935429,  0.02323051,  0.01667355, ...,  0.00638428,
         0.        ,  0.        ],
       [-0.00949132, -0.00060132,  0.02634004, ...,  0.04302868,
         0.00118084,  0.        ],
       [ 0.01757763,  0.01517454,  0.03182393, ...,  0.02904396,
         0.        ,  0.        ],
       ...,
       [ 0.00756703,  0.04082896,  0.0248953 , ...,  0.02749274,
         0.        ,  0.        ],
       [ 0.01357932, -0.00924617,  0.02497569, ...,  0.02593378,
         0.00798608,  0.00781092],
       [ 0.00491172,  0.01239794, -0.00674349, ...,  0.0236083 ,
         0.00174118,  0.00173017]])

In [502]:
y=list(df.iloc[:,-1])
y_train=y[:int(len(cembList)*0.8)]
y_test=y[int(len(cembList)*0.8):int(len(cembList))]
print(y_test)

[1, 1, 1, 0, 2, 0, 0, 1, 2, 0, 1, 1, 1, 2, 1, 0, 1, 1, 1, 1, 2, 0, 1, 1, 1, 1, 1, 1, 2, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 2, 0, 0, 1, 0, 1, 1, 1, 2, 0, 0, 2, 2, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 2, 1, 1, 1, 2, 0, 1, 1, 0, 1, 2, 2, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 2, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 2, 0, 0, 1, 1, 1, 1, 0, 2, 1, 1, 0, 0, 1, 1, 1, 2, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 2, 2, 1, 1, 1, 2, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 2, 1, 1, 2, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 2, 1, 0, 2, 0, 1, 0, 1, 1, 1, 1, 1, 1, 2, 1, 0, 1, 0, 0, 2, 1, 1, 2, 1, 1, 1, 1, 2, 1, 2, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 2, 0, 1, 0, 1, 0, 1, 2, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 2, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 2, 1, 1, 1, 1, 0, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 0, 2, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 2, 0, 0, 1, 2, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 2, 0, 1, 1, 1, 1, 1, 0, 1, 1, 2, 

In [503]:
X_train_embed=cembList[:int(len(cembList)*0.8)]
X_test_embed=cembList[int(len(cembList)*0.8):]

In [504]:
X_test_embed = X_test_embed.astype(np.float32)
X_train_embed = X_train_embed.astype(np.float32)

In [505]:
len(X_test_embed)

1366

In [506]:
dataweights=torch.tensor(cembList)

In [507]:
class MultiModal(nn.Module): #For images
    def __init__(self):
        super(MultiModal, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(1792, 1048),
            nn.ReLU(),
            nn.Linear(1048, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Softmax(),
            nn.Linear(512, 3),
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [508]:
model=MultiModal()

In [509]:
def train_loop(X,y, model, loss_fn, optimizer,batch_size=2):
    size = len(X)
    num_batches = len(X)//batch_size
    prev_batch=0
    lastbatch,lastbatch_size=False,0
    for batch in range(num_batches):
        pred = model(X[prev_batch:prev_batch+batch_size])
        print(pred)
        loss = loss_fn(pred, y[prev_batch:prev_batch+batch_size])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss, current = loss.item(), prev_batch+batch_size
        print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
        prev_batch=prev_batch+batch_size


def test_loop(X,y, model, loss_fn,batch_size=2):
    size = len(X)
    num_batches = size//batch_size
    test_loss, correct, correct2 = 0, 0, 0
    prev_batch=0
    with torch.no_grad():
        for batch in range(num_batches):
            pred = model(X[prev_batch:prev_batch+batch_size])
            test_loss += loss_fn(pred, y[prev_batch:prev_batch+batch_size]).item()
            correct += (pred.argmax(1) == y[prev_batch:prev_batch+batch_size]).type(torch.float).sum().item()
            prev_batch=prev_batch+batch_size
    test_loss /= num_batches
    acc=correct/ size
    print(f"Test Error: \n Accuracy: {(100*acc):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [510]:
learning_rate = 1e-3
batch_size = 2
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
epochs = 1
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(torch.tensor(X_train_embed),torch.tensor(y_train), model, loss_fn, optimizer,batch_size)
    test_loop(torch.tensor(X_test_embed),torch.tensor(y_test), model, loss_fn,batch_size)

Epoch 1
-------------------------------
tensor([[-0.0093,  0.0362,  0.0307],
        [-0.0093,  0.0362,  0.0307]], grad_fn=<AddmmBackward0>)
loss: 1.081795  [    2/ 5464]
tensor([[-0.0114,  0.0383,  0.0286],
        [-0.0114,  0.0383,  0.0286]], grad_fn=<AddmmBackward0>)
loss: 1.079047  [    4/ 5464]
tensor([[-0.0134,  0.0404,  0.0265],
        [-0.0134,  0.0404,  0.0266]], grad_fn=<AddmmBackward0>)
loss: 1.076291  [    6/ 5464]
tensor([[-0.0155,  0.0426,  0.0244],
        [-0.0155,  0.0426,  0.0244]], grad_fn=<AddmmBackward0>)
loss: 1.073465  [    8/ 5464]


C:\Users\MOON COMPUTER\anaconda3\lib\site-packages\torch\nn\modules\container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


tensor([[-0.0176,  0.0449,  0.0223],
        [-0.0176,  0.0449,  0.0223]], grad_fn=<AddmmBackward0>)
loss: 1.101819  [   10/ 5464]
tensor([[-0.0191,  0.0472,  0.0200],
        [-0.0190,  0.0470,  0.0201]], grad_fn=<AddmmBackward0>)
loss: 1.067896  [   12/ 5464]
tensor([[-0.0208,  0.0496,  0.0176],
        [-0.0207,  0.0496,  0.0177]], grad_fn=<AddmmBackward0>)
loss: 1.100112  [   14/ 5464]
tensor([[-0.0220,  0.0521,  0.0151],
        [-0.0219,  0.0520,  0.0151]], grad_fn=<AddmmBackward0>)
loss: 1.062102  [   16/ 5464]
tensor([[-0.0233,  0.0546,  0.0126],
        [-0.0234,  0.0547,  0.0124]], grad_fn=<AddmmBackward0>)
loss: 1.098023  [   18/ 5464]
tensor([[-0.0246,  0.0583,  0.0090],
        [-0.0245,  0.0577,  0.0094]], grad_fn=<AddmmBackward0>)
loss: 1.096493  [   20/ 5464]
tensor([[-0.0252,  0.0608,  0.0061],
        [-0.0251,  0.0600,  0.0067]], grad_fn=<AddmmBackward0>)
loss: 1.095728  [   22/ 5464]
tensor([[-0.0256,  0.0641,  0.0022],
        [-0.0255,  0.0638,  0.0025]], grad_fn=

tensor([[ 0.0003,  0.1577, -0.1088],
        [ 0.0003,  0.1577, -0.1088]], grad_fn=<AddmmBackward0>)
loss: 1.096566  [  138/ 5464]
tensor([[ 0.0006,  0.1586, -0.1103],
        [ 0.0006,  0.1586, -0.1103]], grad_fn=<AddmmBackward0>)
loss: 1.041435  [  140/ 5464]
tensor([[ 0.0010,  0.1594, -0.1119],
        [ 0.0010,  0.1594, -0.1119]], grad_fn=<AddmmBackward0>)
loss: 1.040797  [  142/ 5464]
tensor([[ 0.0015,  0.1602, -0.1136],
        [ 0.0015,  0.1602, -0.1136]], grad_fn=<AddmmBackward0>)
loss: 1.177032  [  144/ 5464]
tensor([[ 0.0021,  0.1608, -0.1149],
        [ 0.0021,  0.1608, -0.1149]], grad_fn=<AddmmBackward0>)
loss: 1.098086  [  146/ 5464]
tensor([[ 0.0024,  0.1613, -0.1159],
        [ 0.0024,  0.1613, -0.1159]], grad_fn=<AddmmBackward0>)
loss: 0.959720  [  148/ 5464]
tensor([[ 0.0025,  0.1622, -0.1171],
        [ 0.0025,  0.1622, -0.1171]], grad_fn=<AddmmBackward0>)
loss: 0.958883  [  150/ 5464]
tensor([[ 0.0023,  0.1632, -0.1183],
        [ 0.0023,  0.1632, -0.1183]], grad_fn=

loss: 0.901306  [  262/ 5464]
tensor([[-0.0048,  0.2207, -0.1915],
        [-0.0048,  0.2207, -0.1915]], grad_fn=<AddmmBackward0>)
loss: 1.013029  [  264/ 5464]
tensor([[-0.0050,  0.2217, -0.1928],
        [-0.0050,  0.2218, -0.1928]], grad_fn=<AddmmBackward0>)
loss: 0.899269  [  266/ 5464]
tensor([[-0.0053,  0.2230, -0.1941],
        [-0.0053,  0.2230, -0.1941]], grad_fn=<AddmmBackward0>)
loss: 1.126379  [  268/ 5464]
tensor([[-0.0052,  0.2239, -0.1955],
        [-0.0052,  0.2239, -0.1955]], grad_fn=<AddmmBackward0>)
loss: 1.011749  [  270/ 5464]
tensor([[-0.0050,  0.2247, -0.1969],
        [-0.0050,  0.2247, -0.1969]], grad_fn=<AddmmBackward0>)
loss: 1.011243  [  272/ 5464]
tensor([[-0.0047,  0.2255, -0.1984],
        [-0.0047,  0.2255, -0.1984]], grad_fn=<AddmmBackward0>)
loss: 1.010708  [  274/ 5464]
tensor([[-0.0043,  0.2263, -0.2000],
        [-0.0043,  0.2263, -0.2000]], grad_fn=<AddmmBackward0>)
loss: 1.010149  [  276/ 5464]
tensor([[-0.0038,  0.2270, -0.2016],
        [-0.0038

...

loss: 0.848587  [ 4844/ 5464]
tensor([[ 0.0709,  0.7451, -1.0983],
        [ 0.0709,  0.7451, -1.0983]], grad_fn=<AddmmBackward0>)
loss: 0.848624  [ 4846/ 5464]
tensor([[ 0.0709,  0.7455, -1.0989],
        [ 0.0709,  0.7455, -1.0989]], grad_fn=<AddmmBackward0>)
loss: 1.185922  [ 4848/ 5464]
tensor([[ 0.0713,  0.7455, -1.0995],
        [ 0.0713,  0.7455, -1.0995]], grad_fn=<AddmmBackward0>)
loss: 0.511359  [ 4850/ 5464]
tensor([[ 0.0714,  0.7458, -1.1002],
        [ 0.0714,  0.7458, -1.1002]], grad_fn=<AddmmBackward0>)
loss: 1.185599  [ 4852/ 5464]
tensor([[ 0.0720,  0.7457, -1.1009],
        [ 0.0720,  0.7457, -1.1009]], grad_fn=<AddmmBackward0>)
loss: 0.511395  [ 4854/ 5464]
tensor([[ 0.0723,  0.7458, -1.1016],
        [ 0.0723,  0.7458, -1.1016]], grad_fn=<AddmmBackward0>)
loss: 0.511376  [ 4856/ 5464]
tensor([[ 0.0724,  0.7462, -1.1023],
        [ 0.0724,  0.7462, -1.1023]], grad_fn=<AddmmBackward0>)
loss: 0.511198  [ 4858/ 5464]
tensor([[ 0.0723,  0.7467, -1.1030],
        [ 0.0723

loss: 0.514745  [ 4974/ 5464]
tensor([[ 0.0817,  0.7425, -1.1105],
        [ 0.0817,  0.7425, -1.1105]], grad_fn=<AddmmBackward0>)
loss: 0.514759  [ 4976/ 5464]
tensor([[ 0.0813,  0.7426, -1.1100],
        [ 0.0813,  0.7426, -1.1100]], grad_fn=<AddmmBackward0>)
loss: 1.771581  [ 4978/ 5464]
tensor([[ 0.0809,  0.7424, -1.1091],
        [ 0.0809,  0.7424, -1.1091]], grad_fn=<AddmmBackward0>)
loss: 0.514688  [ 4980/ 5464]
tensor([[ 0.0805,  0.7424, -1.1084],
        [ 0.0805,  0.7424, -1.1084]], grad_fn=<AddmmBackward0>)
loss: 1.440009  [ 4982/ 5464]
tensor([[ 0.0798,  0.7424, -1.1074],
        [ 0.0798,  0.7424, -1.1074]], grad_fn=<AddmmBackward0>)
loss: 0.514516  [ 4984/ 5464]
tensor([[ 0.0791,  0.7426, -1.1066],
        [ 0.0791,  0.7426, -1.1066]], grad_fn=<AddmmBackward0>)
loss: 0.846012  [ 4986/ 5464]
tensor([[ 0.0785,  0.7427, -1.1060],
        [ 0.0785,  0.7427, -1.1060]], grad_fn=<AddmmBackward0>)
loss: 0.514121  [ 4988/ 5464]
tensor([[ 0.0778,  0.7430, -1.1055],
        [ 0.0778

loss: 0.504491  [ 5102/ 5464]
tensor([[ 0.0636,  0.7561, -1.1053],
        [ 0.0636,  0.7561, -1.1053]], grad_fn=<AddmmBackward0>)
loss: 0.504275  [ 5104/ 5464]
tensor([[ 0.0630,  0.7565, -1.1053],
        [ 0.0630,  0.7565, -1.1053]], grad_fn=<AddmmBackward0>)
loss: 0.850682  [ 5106/ 5464]
tensor([[ 0.0627,  0.7569, -1.1054],
        [ 0.0627,  0.7569, -1.1054]], grad_fn=<AddmmBackward0>)
loss: 0.503662  [ 5108/ 5464]
tensor([[ 0.0622,  0.7575, -1.1056],
        [ 0.0622,  0.7575, -1.1056]], grad_fn=<AddmmBackward0>)
loss: 0.503271  [ 5110/ 5464]
tensor([[ 0.0615,  0.7582, -1.1058],
        [ 0.0615,  0.7582, -1.1058]], grad_fn=<AddmmBackward0>)
loss: 1.434777  [ 5112/ 5464]
tensor([[ 0.0608,  0.7588, -1.1056],
        [ 0.0608,  0.7588, -1.1056]], grad_fn=<AddmmBackward0>)
loss: 1.783567  [ 5114/ 5464]
tensor([[ 0.0602,  0.7590, -1.1051],
        [ 0.0602,  0.7590, -1.1051]], grad_fn=<AddmmBackward0>)
loss: 0.851517  [ 5116/ 5464]
tensor([[ 0.0598,  0.7591, -1.1047],
        [ 0.0598

loss: 0.516448  [ 5232/ 5464]
tensor([[ 0.0745,  0.7371, -1.0906],
        [ 0.0745,  0.7371, -1.0906]], grad_fn=<AddmmBackward0>)
loss: 0.516613  [ 5234/ 5464]
tensor([[ 0.0746,  0.7371, -1.0908],
        [ 0.0746,  0.7371, -1.0908]], grad_fn=<AddmmBackward0>)
loss: 0.847842  [ 5236/ 5464]
tensor([[ 0.0748,  0.7371, -1.0911],
        [ 0.0748,  0.7371, -1.0911]], grad_fn=<AddmmBackward0>)
loss: 0.516639  [ 5238/ 5464]
tensor([[ 0.0748,  0.7373, -1.0914],
        [ 0.0748,  0.7373, -1.0914]], grad_fn=<AddmmBackward0>)
loss: 0.847761  [ 5240/ 5464]
tensor([[ 0.0750,  0.7375, -1.0919],
        [ 0.0750,  0.7375, -1.0919]], grad_fn=<AddmmBackward0>)
loss: 0.516451  [ 5242/ 5464]
tensor([[ 0.0749,  0.7378, -1.0923],
        [ 0.0749,  0.7378, -1.0923]], grad_fn=<AddmmBackward0>)
loss: 0.516233  [ 5244/ 5464]
tensor([[ 0.0746,  0.7384, -1.0928],
        [ 0.0746,  0.7384, -1.0928]], grad_fn=<AddmmBackward0>)
loss: 0.515877  [ 5246/ 5464]
tensor([[ 0.0742,  0.7391, -1.0934],
        [ 0.0742

loss: 0.508594  [ 5358/ 5464]
tensor([[ 0.0638,  0.7494, -1.0947],
        [ 0.0638,  0.7494, -1.0947]], grad_fn=<AddmmBackward0>)
loss: 0.508011  [ 5360/ 5464]
tensor([[ 0.0629,  0.7503, -1.0950],
        [ 0.0629,  0.7503, -1.0950]], grad_fn=<AddmmBackward0>)
loss: 0.851032  [ 5362/ 5464]
tensor([[ 0.0623,  0.7512, -1.0954],
        [ 0.0623,  0.7512, -1.0954]], grad_fn=<AddmmBackward0>)
loss: 0.851209  [ 5364/ 5464]
tensor([[ 0.0618,  0.7519, -1.0958],
        [ 0.0618,  0.7519, -1.0958]], grad_fn=<AddmmBackward0>)
loss: 0.851330  [ 5366/ 5464]
tensor([[ 0.0615,  0.7524, -1.0963],
        [ 0.0615,  0.7524, -1.0963]], grad_fn=<AddmmBackward0>)
loss: 0.505962  [ 5368/ 5464]
tensor([[ 0.0611,  0.7531, -1.0968],
        [ 0.0611,  0.7531, -1.0968]], grad_fn=<AddmmBackward0>)
loss: 0.851516  [ 5370/ 5464]
tensor([[ 0.0608,  0.7537, -1.0974],
        [ 0.0608,  0.7537, -1.0974]], grad_fn=<AddmmBackward0>)
loss: 1.430654  [ 5372/ 5464]
tensor([[ 0.0603,  0.7542, -1.0975],
        [ 0.0603

In [511]:
!pip install torchviz

  Created wheel for torchviz: filename=torchviz-0.0.2-py3-none-any.whl size=4151 sha256=25e4815f5ffe73094584ce52b095e8022264684c519dcdfa0eadf8ac71ed1c72
  Stored in directory: c:\users\moon computer\appdata\local\pip\cache\wheels\05\7d\1b\8306781244e42ede119edbb053bdcda1c1f424ca226165a417
Successfully built torchviz


In [ ]:
Y=model(X)

In [ ]:
from torchviz import make_dot
make_dot(Y.mean(), params=dict(model.named_parameters())).render("attached", format="png")